In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build
import exafmm as fmm
import exafmm.laplace as laplace
import exafmm.helmholtz as helmholtz
import numpy as np

In [2]:
print(fmm.__doc__)
print(laplace.__doc__)
print(helmholtz.__doc__)

exafmm's pybind11 module
exafmm's submodule for Laplace kernel
exafmm's submodule for Helmholtz kernel


#### create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

In [4]:
src_coords = np.random.random((nsrcs, 3))
src_charges = np.random.random(nsrcs)
trg_coords = np.random.random((ntrgs, 3))

sources_l = laplace.init_sources(src_coords, src_charges)
targets_l = laplace.init_targets(trg_coords)

In [5]:
src_charges = np.zeros(nsrcs, dtype=np.complex_)
src_charges.real = np.random.random(nsrcs)
src_charges.imag = np.random.random(nsrcs)

sources_h = helmholtz.init_sources(src_coords, src_charges)
targets_h = helmholtz.init_targets(trg_coords)

#### creat FMM instances, set FMM parameters

In [6]:
fmm_l = laplace.LaplaceFMM()
fmm_l.p = 10
fmm_l.nsurf = 6*(fmm_l.p-1)*(fmm_l.p-1) + 2
fmm_l.ncrit = 400
fmm_l.depth = 4

In [7]:
fmm_h = helmholtz.HelmholtzFMM()
fmm_h.p = 10
fmm_h.nsurf = 6*(fmm_h.p-1)*(fmm_h.p-1) + 2
fmm_h.ncrit = 400
fmm_h.depth = 4
fmm_h.wavek = 10

#### build tree

In [8]:
tree_l = laplace.build_tree(sources_l, targets_l, fmm_l)

In [9]:
print(fmm_l.r0)  # root node radius
print(fmm_l.x0)  # root node coords

0.5000011377172792
[0.500000, 0.500000, 0.499995]


In [10]:
tree_l.leafs[5].level

4

In [11]:
tree_h = helmholtz.build_tree(sources_h, targets_h, fmm_h)

In [12]:
print(fmm_h.r0)  # root node radius
print(fmm_h.x0)  # root node coords

0.5000011377172792
[0.500000, 0.500000, 0.499995]


In [13]:
type(tree_l)

exafmm.laplace.Tree

#### build interaction lists

In [14]:
fmm.init_rel_coord()  # initialize relative coordinates matrix

In [15]:
skip_P2P = False   # flag to skip near-field evaluation
laplace.build_list(tree_l, fmm_l, skip_P2P)
helmholtz.build_list(tree_h, fmm_h, skip_P2P)

#### M2L setup

In [16]:
fmm_l.M2L_setup(tree_l.nonleafs)

In [17]:
fmm_h.M2L_setup(tree_h.nonleafs)

#### pre-compute

In [18]:
fmm_l.precompute()

In [19]:
fmm_h.precompute()

#### evaluate potential

In [20]:
trg_value_l = laplace.evaluate(tree_l, fmm_l)

P2M                  : 1.7733800e-01
M2M                  : 8.7459000e-02
P2L                  : 1.4790000e-03
M2P                  : 1.3840000e-03
P2P                  : 4.3667000e-02
M2L                  : 6.5973100e-01
L2L                  : 8.1354000e-02
L2P                  : 1.3205900e-01


`evaluate()` now return an numpy array with shape `(n_trg, 4)`. The i-th row consists of the potential and 3 components of the gradients of the i-th target.

In [22]:
trg_value_l.shape

(100000, 4)

In [23]:
trg_value_h = helmholtz.evaluate(tree_h, fmm_h)

P2M                  : 9.6879100e-01
M2M                  : 4.0530400e-01
P2L                  : 1.8750000e-03
M2P                  : 3.0500000e-03
P2P                  : 4.5176700e-01
M2L                  : 1.7441870e+00
L2L                  : 3.5785500e-01
L2P                  : 9.3800000e-01


In [25]:
print(trg_value_h.shape)
print(trg_value_h[50])

(100000, 4)
[-1.35664165e+03 +381.74668136j -1.27717456e+03+3082.11452063j
 -3.61940931e-01-1593.31474653j  3.26362743e+03-2628.04822698j]


#### check accuracy

For now, this is only valid when skip_P2P is disabled.

In [26]:
fmm_l.verify(tree_l.leafs)

[3.07434641599553e-10, 8.223430699980925e-08]

In [27]:
fmm_h.verify(tree_h.leafs)

[7.531061595344642e-09, 2.9993489913000785e-07]

#### update charges and run FMM iteratively (will be fixed later)

"""
niters = 10
for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    src_charges = np.random.random(nsrcs)   # generate new random charges
    fmm.update(src_charges)      # update charges
    fmm.clear()                  # clear values
    potentials = fmm.evaluate()  # evaluate potentials
    fmm.check_accuracy()         # check accuracy
"""